[Homework for AI-For-Beginners course: 03-Perceptron](https://github.com/microsoft/AI-For-Beginners/blob/main/lessons/3-NeuralNetworks/03-Perceptron/lab/PerceptronMultiClass.ipynb)

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# %pip install datasets
# %pip install numpy

In [ ]:
# display all outputs from Jupyter notebook cells, not just last.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datasets import load_dataset
import torch
import numpy as np

# pick the seed for reproducability - change it to explore the effects of random variations
np.random.seed(1)
import random

In [2]:
dataset_train = load_dataset("mnist", split="train[:10%]")
dataset_label = dataset_train.features['label']
dataset_label_as_num = [dataset_label.str2int(label_name) for label_name in dataset_label.names]

In [11]:
class PerceptronMultiClassMNIST:
  num_iterations = 1000
  learning_rate = 0.01

  # Labels are the numbers from 0 to 9.
  # For each label, split the dataset into two parts: one with the current label and one with all other labels.
  def set_mnist_one_vs_other(dataset_train, label):
    current_images = dataset_train.filter(lambda example: example["label"] == label)
    other_images = dataset_train.filter(lambda example: example["label"] != label)
    return current_images, other_images

  # Train a perceptron to distinguish between the current label and all other labels.
  def train(positive_examples, negative_examples):
    weights = np.zeros((28,28))

    for _ in range(PerceptronMultiClassMNIST.num_iterations):
        pos = np.asarray(random.choice(positive_examples)['image'])
        neg = np.asarray(random.choice(negative_examples)['image'])

        if np.sum(np.dot(pos, weights)) <= 0:
            weights += pos

        if np.sum(np.dot(neg, weights)) >= 0:
            weights -= neg

    return weights
  
  # Same but with learning rate
  def train_lr(positive_examples, negative_examples):
    weights = np.zeros((28,28))

    for _ in range(PerceptronMultiClassMNIST.num_iterations):
        pos = np.asarray(random.choice(positive_examples)['image'])
        neg = np.asarray(random.choice(negative_examples)['image'])

        if np.sum(np.dot(pos, weights)) <= 0:
            weights += PerceptronMultiClassMNIST.learning_rate * pos

        if np.sum(np.dot(neg, weights)) >= 0:
            weights -= PerceptronMultiClassMNIST.learning_rate * neg

    return weights

  def sigmoid(x):
      return 1 / (1 + np.exp(-x))

  def normalize(image):
      return (image - np.mean(image)) / np.std(image)

  # Same but with sigmoid
  def train_lr_sigmoid(positive_examples, negative_examples):
    weights = np.zeros((28,28))

    for _ in range(PerceptronMultiClassMNIST.num_iterations):
        pos = PerceptronMultiClassMNIST.normalize(np.asarray(random.choice(positive_examples)['image']))
        neg = PerceptronMultiClassMNIST.normalize(np.asarray(random.choice(negative_examples)['image']))

        pos_error = 1 - PerceptronMultiClassMNIST.sigmoid(np.sum(np.dot(pos, weights)))
        neg_error = PerceptronMultiClassMNIST.sigmoid(np.sum(np.dot(neg, weights)))

        weights += PerceptronMultiClassMNIST.learning_rate * pos_error * pos
        weights -= PerceptronMultiClassMNIST.learning_rate * neg_error * neg

    return weights

  # Train a perceptron for each label.
  def train_perceptrons(dataset_train, dataset_label_as_num, train_fn):
    perceptrons = np.zeros(10, dtype=object)
    for label_number in dataset_label_as_num:
      positive_label, other_labels = PerceptronMultiClassMNIST.set_mnist_one_vs_other(dataset_train, label_number)
      perceptrons[label_number] = train_fn(positive_label, other_labels)
    return perceptrons

In [ ]:
perceptrons = PerceptronMultiClassMNIST.train_perceptrons(dataset_train, dataset_label_as_num, PerceptronMultiClassMNIST.train)

In [54]:
def predict(image, weights):
    z = np.sum(np.dot(image, weights))
    return 1 if z > 0.9 else 0

def classify(perceptrons, image):
    predictions = [predict(image, weights) for weights in perceptrons]
    return np.argmax(predictions), predictions

def accuracy(dataset, perceptrons):
    correct = 0
    for example in dataset:
        label = example['label']
        prediction, _ = classify(perceptrons, example['image'])
        if prediction == label:
            correct += 1
    return correct / len(dataset)

In [57]:
digit = random.choice(dataset_train)
actual_digit = digit['label']
predicted_digit, all_predictions = classify(perceptrons, digit['image'])

all_predictions
actual_digit
predicted_digit
actual_digit == predicted_digit
digit['image']

[0, 1, 0, 0, 1, 0, 1, 1, 0, 1]

8

1

False

In [55]:
accuracy(dataset_train, perceptrons)

0.13333333333333333